### _Extract Data_

In [1]:
import requests
import pandas as pd
import firebase_admin
from firebase_admin import storage
from firebase_admin import credentials

In [2]:
def extract_data() -> list:
    url = "https://gist.githubusercontent.com/nadirbslmh/8fc9cc6cd5cbaaf5cbff63b090fb497e/raw/a7bf3e1edab88b04314a40a9de3ed744bc86d0e9/ecommerce.json"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Failed to fetch data from the API")
        return []

### _Transform Data_

In [6]:
def transform_data() -> pd.DataFrame:
    data = extract_data()
    df = pd.json_normalize(data)
    filtered_df = df[(df['payment_method'] == 'credit card') & (df['status'] == 'success')]
    filtered_df.reset_index(drop=True, inplace=True)
    return filtered_df

transform_data()

,transaction_id,transaction_date,customer_name,transaction_amount,item_category,item_name,payment_method,status
0,1,01-11-2023,John Doe,50.0,book,The Great Gatsby,credit card,success
1,5,03-11-2023,Charlie Brown,150.0,book,To Kill a Mockingbird,credit card,success
2,9,05-11-2023,Olivia Martin,350.0,book,1984,credit card,success
3,12,07-11-2023,William Turner,18.0,kitchen,Toaster,credit card,success
4,16,10-11-2023,Lily Anderson,8.0,kitchen,Kitchen Scale,credit card,success
5,17,10-11-2023,Mason Evans,280.0,book,The Catcher in the Rye,credit card,success
6,20,12-11-2023,Isabella Martinez,35.0,kitchen,Cookware Set,credit card,success
7,21,12-11-2023,Noah Clark,9.0,book,Lord of the Rings Trilogy,credit card,success
8,25,15-11-2023,Ava Jones,17.0,book,The Hobbit,credit card,success
9,27,16-11-2023,Sophie Davis,500.0,technology,Laptop,credit card,success


### _Load Data_

In [8]:
def load_data():
    try:
        data = transform_data()
        cred = credentials.Certificate("serviceAccountKey.json")
        storage_bucket = "project-1-418609.appspot.com"
        firebase_admin.initialize_app(cred, {"storageBucket": storage_bucket})
        bucket = storage.bucket()
                
        file_path = "data_source/"
        file_name = "e-commerce.csv"
        data.to_csv(file_path + file_name, index=False)
                
        blob = bucket.blob(file_name)
        blob.upload_from_filename(file_path + file_name)
                
        print(f"Data has been successfully loaded to the storage")
    except Exception as e:
        print(f"An error occurred when loading data to the storage: {e}")
load_data()

Data has been successfully loaded to the storage
